# Demographic Inference Analysis

Analysis of LLM-inferred demographics from YouTube commenter profiles.

This notebook examines:
- Distribution of inferred age ranges
- Distribution of inferred gender
- Distribution of inferred race/ethnicity
- Confidence level distributions
- Inference quality ("unclear" rates)
- Reasoning patterns

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import glob
import os

print("Libraries loaded successfully")

Libraries loaded successfully


## Load Data

In [2]:
# Load the most recent user demographics CSV
csv_files = glob.glob("data/user_demographics_*.csv")
if not csv_files:
	raise FileNotFoundError("No user demographics CSV files found in data/ directory")

demographics_file = max(csv_files, key=os.path.getmtime)
print(f"Loading: {demographics_file}")

df = pd.read_csv(demographics_file)
print(f"\nLoaded {len(df)} user profiles with demographic inferences")
print(f"\nColumns: {list(df.columns)}")

Loading: data/user_demographics_20251030_0843.csv

Loaded 99 user profiles with demographic inferences

Columns: ['channel_id', 'channel_title', 'thumbnail_url', 'channel_description', 'channel_country', 'inferred_age_range', 'inferred_gender', 'inferred_race_ethnicity', 'confidence_level', 'reasoning']


In [3]:
# Display first few rows
df.head()

,channel_id,channel_title,thumbnail_url,channel_description,channel_country,inferred_age_range,inferred_gender,inferred_race_ethnicity,confidence_level,reasoning
0,UCInySXd_eU3qLargrnaUFtQ,Zero Heat,https://yt3.ggpht.com/ytc/AIdro_nqJzqrX1gwmwtB...,NaN,NaN,unclear,unclear,unclear,0.1,The profile image shows a non-human graphic (l...
1,UCl8o-Cf3BVTUh-FF58vYwYw,Beavis Kornholio,https://yt3.ggpht.com/ytc/AIdro_kpdLQ5DAEj0Haj...,NaN,NaN,unclear,unclear,unclear,0.1,The profile image is a generic letter 'B' on a...
2,UCyY5co_QOWj8zrKXpVH4m1Q,Byun Baekhyun,https://yt3.ggpht.com/ytc/AIdro_l8e95GPr75xfck...,hi there! how are you guys doing? wish you guy...,NaN,18-24,unclear,asian,0.7,The username 'Byun Baekhyun' and the content r...
3,UC3Ll2JweKwRerN9iPm-Q1xw,merseybeat1963,https://yt3.ggpht.com/ytc/AIdro_kdrvYms_C3sYi5...,NaN,NaN,55-64,unclear,unclear,0.6,The username 'merseybeat1963' suggests an affi...
4,UCJ69vnObSmzJOxlCe8tmQHw,FocusJay_,https://yt3.ggpht.com/mEJf4XJ2ONG-ErCfz_t7f72F...,What's up you guys?! Welcome to my channel. A...,NaN,unclear,unclear,unclear,0.2,The profile image does not clearly show a pers...


In [4]:
# Basic statistics
print("Dataset Overview:")
print(f"Total users analyzed: {len(df)}")
print(f"\nMissing values:")
print(df.isnull().sum())
print(f"\nConfidence level statistics:")
print(df['confidence_level'].describe())

Dataset Overview:
Total users analyzed: 99

Missing values:
channel_id                  0
channel_title               0
thumbnail_url               0
channel_description        80
channel_country            90
inferred_age_range          0
inferred_gender             0
inferred_race_ethnicity     0
confidence_level            0
reasoning                   0
dtype: int64

Confidence level statistics:
count    99.000000
mean      0.326768
std       0.271927
min       0.100000
25%       0.100000
50%       0.200000
75%       0.600000
max       0.900000
Name: confidence_level, dtype: float64


## Age Distribution

In [5]:
# Age range distribution
age_counts = df['inferred_age_range'].value_counts().sort_index()
print("Age Range Distribution:")
print(age_counts)
print(f"\nPercentages:")
print((age_counts / len(df) * 100).round(2))

Age Range Distribution:
inferred_age_range
18-24       4
25-34       6
35-44       5
45-54       2
55-64       2
unclear    80
Name: count, dtype: int64

Percentages:
inferred_age_range
18-24       4.04
25-34       6.06
35-44       5.05
45-54       2.02
55-64       2.02
unclear    80.81
Name: count, dtype: float64


In [6]:
# Plot age distribution
age_order = ['under_18', '18-24', '25-34', '35-44', '45-54', '55-64', '65_plus', 'unclear']
age_data = df['inferred_age_range'].value_counts().reindex(age_order, fill_value=0)

fig = go.Figure(data=[
	go.Bar(
		x=age_data.index,
		y=age_data.values,
		text=age_data.values,
		textposition='outside',
		marker_color='steelblue'
	)
])

fig.update_layout(
	title='Distribution of Inferred Age Ranges',
	xaxis_title='Age Range',
	yaxis_title='Count',
	height=500
)

fig.show()

# Calculate unclear rate
unclear_rate = (df['inferred_age_range'] == 'unclear').sum() / len(df) * 100
print(f"\nAge 'unclear' rate: {unclear_rate:.2f}%")


Age 'unclear' rate: 80.81%


## Gender Distribution

In [7]:
# Gender distribution
gender_counts = df['inferred_gender'].value_counts()
print("Gender Distribution:")
print(gender_counts)
print(f"\nPercentages:")
print((gender_counts / len(df) * 100).round(2))

Gender Distribution:
inferred_gender
unclear    60
male       32
female      7
Name: count, dtype: int64

Percentages:
inferred_gender
unclear    60.61
male       32.32
female      7.07
Name: count, dtype: float64


In [8]:
# Plot gender distribution
gender_order = ['male', 'female', 'non_binary', 'unclear']
gender_data = df['inferred_gender'].value_counts().reindex(gender_order, fill_value=0)

# Create subplots
fig = make_subplots(
	rows=1, cols=2,
	subplot_titles=('Distribution of Inferred Gender', 'Gender Distribution (Excluding "Unclear")'),
	specs=[[{'type': 'bar'}, {'type': 'pie'}]]
)

# Bar chart
fig.add_trace(
	go.Bar(
		x=gender_data.index,
		y=gender_data.values,
		text=gender_data.values,
		textposition='outside',
		marker_color='coral',
		name='Count'
	),
	row=1, col=1
)

# Pie chart (excluding unclear)
gender_clear = df[df['inferred_gender'] != 'unclear']['inferred_gender'].value_counts()
if len(gender_clear) > 0:
	fig.add_trace(
		go.Pie(
			labels=gender_clear.index,
			values=gender_clear.values,
			name='Gender'
		),
		row=1, col=2
	)

fig.update_xaxes(title_text='Gender', row=1, col=1)
fig.update_yaxes(title_text='Count', row=1, col=1)
fig.update_layout(height=500, showlegend=False)

fig.show()

# Calculate unclear rate
unclear_rate = (df['inferred_gender'] == 'unclear').sum() / len(df) * 100
print(f"\nGender 'unclear' rate: {unclear_rate:.2f}%")


Gender 'unclear' rate: 60.61%


## Race/Ethnicity Distribution

In [9]:
# Race/ethnicity distribution
race_counts = df['inferred_race_ethnicity'].value_counts()
print("Race/Ethnicity Distribution:")
print(race_counts)
print(f"\nPercentages:")
print((race_counts / len(df) * 100).round(2))

Race/Ethnicity Distribution:
inferred_race_ethnicity
unclear                   89
black_african_american     5
hispanic_latino            3
asian                      1
white                      1
Name: count, dtype: int64

Percentages:
inferred_race_ethnicity
unclear                   89.90
black_african_american     5.05
hispanic_latino            3.03
asian                      1.01
white                      1.01
Name: count, dtype: float64


In [10]:
# Plot race/ethnicity distribution
race_data = df['inferred_race_ethnicity'].value_counts().sort_values(ascending=True)

fig = go.Figure(data=[
	go.Bar(
		y=race_data.index,
		x=race_data.values,
		text=race_data.values,
		textposition='outside',
		orientation='h',
		marker_color='seagreen'
	)
])

fig.update_layout(
	title='Distribution of Inferred Race/Ethnicity',
	xaxis_title='Count',
	yaxis_title='Race/Ethnicity',
	height=600
)

fig.show()

# Calculate unclear rate
unclear_rate = (df['inferred_race_ethnicity'] == 'unclear').sum() / len(df) * 100
print(f"\nRace/ethnicity 'unclear' rate: {unclear_rate:.2f}%")


Race/ethnicity 'unclear' rate: 89.90%


## Confidence Level Analysis

In [11]:
# Confidence level distribution
print("Confidence Level Statistics:")
print(df['confidence_level'].describe())
print(f"\nConfidence ranges:")
print(f"Very low (0.0-0.2): {((df['confidence_level'] <= 0.2).sum() / len(df) * 100):.2f}%")
print(f"Low (0.2-0.4): {(((df['confidence_level'] > 0.2) & (df['confidence_level'] <= 0.4)).sum() / len(df) * 100):.2f}%")
print(f"Medium (0.4-0.6): {(((df['confidence_level'] > 0.4) & (df['confidence_level'] <= 0.6)).sum() / len(df) * 100):.2f}%")
print(f"Medium-High (0.6-0.8): {(((df['confidence_level'] > 0.6) & (df['confidence_level'] <= 0.8)).sum() / len(df) * 100):.2f}%")
print(f"High (0.8-1.0): {((df['confidence_level'] > 0.8).sum() / len(df) * 100):.2f}%")

Confidence Level Statistics:
count    99.000000
mean      0.326768
std       0.271927
min       0.100000
25%       0.100000
50%       0.200000
75%       0.600000
max       0.900000
Name: confidence_level, dtype: float64

Confidence ranges:
Very low (0.0-0.2): 56.57%
Low (0.2-0.4): 8.08%
Medium (0.4-0.6): 16.16%
Medium-High (0.6-0.8): 14.14%
High (0.8-1.0): 5.05%


In [12]:
# Plot confidence distribution
fig = make_subplots(
	rows=1, cols=2,
	subplot_titles=('Distribution of Confidence Levels', 'Confidence Level Box Plot')
)

# Histogram
fig.add_trace(
	go.Histogram(
		x=df['confidence_level'],
		nbinsx=20,
		marker_color='mediumpurple',
		name='Count'
	),
	row=1, col=1
)

# Add mean and median lines
mean_conf = df['confidence_level'].mean()
median_conf = df['confidence_level'].median()

fig.add_vline(x=mean_conf, line_dash="dash", line_color="red", 
			  annotation_text=f"Mean: {mean_conf:.2f}",
			  annotation_position="top right", row=1, col=1)
fig.add_vline(x=median_conf, line_dash="dash", line_color="orange",
			  annotation_text=f"Median: {median_conf:.2f}",
			  annotation_position="top left", row=1, col=1)

# Box plot
fig.add_trace(
	go.Box(
		y=df['confidence_level'],
		marker_color='mediumpurple',
		name='All Inferences'
	),
	row=1, col=2
)

fig.update_xaxes(title_text='Confidence Level', row=1, col=1)
fig.update_yaxes(title_text='Count', row=1, col=1)
fig.update_yaxes(title_text='Confidence Level', row=1, col=2)
fig.update_layout(height=500, showlegend=False)

fig.show()

## Confidence by Demographic Category

In [13]:
# Confidence by age range
age_confidence = df.groupby('inferred_age_range')['confidence_level'].agg(['mean', 'median', 'count']).sort_values('mean', ascending=False)
print("Confidence by Age Range:")
print(age_confidence)

Confidence by Age Range:
                        mean  median  count
inferred_age_range                         
45-54               0.825000   0.825      2
35-44               0.790000   0.800      5
25-34               0.758333   0.750      6
18-24               0.750000   0.725      4
55-64               0.450000   0.450      2
unclear             0.228750   0.100     80


In [14]:
# Confidence by gender
gender_confidence = df.groupby('inferred_gender')['confidence_level'].agg(['mean', 'median', 'count']).sort_values('mean', ascending=False)
print("Confidence by Gender:")
print(gender_confidence)

Confidence by Gender:
                     mean  median  count
inferred_gender                         
female           0.621429    0.60      7
male             0.615625    0.65     32
unclear          0.138333    0.10     60


In [15]:
# Confidence by race/ethnicity
race_confidence = df.groupby('inferred_race_ethnicity')['confidence_level'].agg(['mean', 'median', 'count']).sort_values('mean', ascending=False)
print("Confidence by Race/Ethnicity:")
print(race_confidence)

Confidence by Race/Ethnicity:
                             mean  median  count
inferred_race_ethnicity                         
white                    0.800000     0.8      1
black_african_american   0.750000     0.8      5
asian                    0.700000     0.7      1
hispanic_latino          0.566667     0.6      3
unclear                  0.285393     0.1     89


In [16]:
# Visualize confidence by category
age_order = ['under_18', '18-24', '25-34', '35-44', '45-54', '55-64', '65_plus', 'unclear']
gender_order = ['male', 'female', 'non_binary', 'unclear']

age_conf_plot = df.groupby('inferred_age_range')['confidence_level'].mean().reindex(age_order, fill_value=0)
gender_conf_plot = df.groupby('inferred_gender')['confidence_level'].mean().reindex(gender_order, fill_value=0)
race_conf_plot = df.groupby('inferred_race_ethnicity')['confidence_level'].mean().sort_values(ascending=False)

fig = make_subplots(
	rows=3, cols=1,
	subplot_titles=('Mean Confidence Level by Age Range', 
					'Mean Confidence Level by Gender',
					'Mean Confidence Level by Race/Ethnicity'),
	vertical_spacing=0.12
)

# Age
fig.add_trace(
	go.Bar(x=age_conf_plot.index, y=age_conf_plot.values, 
		   marker_color='steelblue', showlegend=False),
	row=1, col=1
)

# Gender
fig.add_trace(
	go.Bar(x=gender_conf_plot.index, y=gender_conf_plot.values,
		   marker_color='coral', showlegend=False),
	row=2, col=1
)

# Race/Ethnicity (horizontal)
fig.add_trace(
	go.Bar(y=race_conf_plot.index, x=race_conf_plot.values,
		   orientation='h', marker_color='seagreen', showlegend=False),
	row=3, col=1
)

fig.update_yaxes(title_text='Mean Confidence', range=[0, 1], row=1, col=1)
fig.update_yaxes(title_text='Mean Confidence', range=[0, 1], row=2, col=1)
fig.update_xaxes(title_text='Mean Confidence', range=[0, 1], row=3, col=1)

fig.update_layout(height=1000)
fig.show()

## Overall "Unclear" Rate Analysis

In [17]:
# Calculate unclear rates for each category
age_unclear = (df['inferred_age_range'] == 'unclear').sum()
gender_unclear = (df['inferred_gender'] == 'unclear').sum()
race_unclear = (df['inferred_race_ethnicity'] == 'unclear').sum()

# How many profiles have all three as unclear?
all_unclear = ((df['inferred_age_range'] == 'unclear') & 
			   (df['inferred_gender'] == 'unclear') & 
			   (df['inferred_race_ethnicity'] == 'unclear')).sum()

# How many have at least one clear inference?
at_least_one_clear = ((df['inferred_age_range'] != 'unclear') | 
					  (df['inferred_gender'] != 'unclear') | 
					  (df['inferred_race_ethnicity'] != 'unclear')).sum()

print("Unclear Rate Summary:")
print(f"Age unclear: {age_unclear} ({age_unclear/len(df)*100:.2f}%)")
print(f"Gender unclear: {gender_unclear} ({gender_unclear/len(df)*100:.2f}%)")
print(f"Race/ethnicity unclear: {race_unclear} ({race_unclear/len(df)*100:.2f}%)")
print(f"\nAll three unclear: {all_unclear} ({all_unclear/len(df)*100:.2f}%)")
print(f"At least one clear inference: {at_least_one_clear} ({at_least_one_clear/len(df)*100:.2f}%)")

Unclear Rate Summary:
Age unclear: 80 (80.81%)
Gender unclear: 60 (60.61%)
Race/ethnicity unclear: 89 (89.90%)

All three unclear: 56 (56.57%)
At least one clear inference: 43 (43.43%)


In [18]:
# Plot unclear rates
categories = ['Age', 'Gender', 'Race/Ethnicity', 'All Three', 'At Least One Clear']
unclear_counts = [
	age_unclear,
	gender_unclear,
	race_unclear,
	all_unclear,
	at_least_one_clear
]
colors = ['steelblue', 'coral', 'seagreen', 'gray', 'gold']

# Create text labels with counts and percentages
text_labels = [f'{int(count)}<br>({count/len(df)*100:.1f}%)' for count in unclear_counts]

fig = go.Figure(data=[
	go.Bar(
		x=categories,
		y=unclear_counts,
		text=text_labels,
		textposition='outside',
		marker_color=colors
	)
])

fig.update_layout(
	title='"Unclear" Inference Counts by Category',
	xaxis_title='',
	yaxis_title='Count',
	height=500
)

fig.show()

## Demographic Combinations

In [19]:
# Most common demographic combinations (excluding unclear)
df_clear = df[
	(df['inferred_age_range'] != 'unclear') |
	(df['inferred_gender'] != 'unclear') |
	(df['inferred_race_ethnicity'] != 'unclear')
]

combo_counts = df_clear.groupby(['inferred_gender', 'inferred_age_range', 'inferred_race_ethnicity']).size().sort_values(ascending=False).head(20)
print("Top 20 Demographic Combinations (with at least one clear inference):")
print(combo_counts)

Top 20 Demographic Combinations (with at least one clear inference):
inferred_gender  inferred_age_range  inferred_race_ethnicity
male             unclear             unclear                    15
female           unclear             unclear                     4
male             25-34               unclear                     4
                 35-44               unclear                     3
                                     black_african_american      2
unclear          55-64               unclear                     2
male             18-24               unclear                     2
                 unclear             hispanic_latino             2
                                     black_african_american      1
unclear          18-24               asian                       1
female           25-34               unclear                     1
male             45-54               unclear                     1
female           45-54               unclear                     1

In [20]:
# Gender x Age crosstab
gender_age = pd.crosstab(df['inferred_gender'], df['inferred_age_range'])
print("\nGender x Age Crosstab:")
print(gender_age)


Gender x Age Crosstab:
inferred_age_range  18-24  25-34  35-44  45-54  55-64  unclear
inferred_gender                                               
female                  0      1      0      1      0        5
male                    3      5      5      1      0       18
unclear                 1      0      0      0      2       57


In [21]:
# Visualize gender x age heatmap
age_order = ['under_18', '18-24', '25-34', '35-44', '45-54', '55-64', '65_plus', 'unclear']
gender_order = ['male', 'female', 'non_binary', 'unclear']

gender_age_ordered = pd.crosstab(df['inferred_gender'], df['inferred_age_range'])
gender_age_ordered = gender_age_ordered.reindex(index=gender_order, columns=age_order, fill_value=0)

fig = go.Figure(data=go.Heatmap(
	z=gender_age_ordered.values,
	x=gender_age_ordered.columns,
	y=gender_age_ordered.index,
	colorscale='YlOrRd',
	text=gender_age_ordered.values,
	texttemplate='%{text}',
	textfont={"size": 12},
	colorbar=dict(title="Count")
))

fig.update_layout(
	title='Gender x Age Range Heatmap',
	xaxis_title='Age Range',
	yaxis_title='Gender',
	height=500
)

fig.show()

## Reasoning Analysis

In [22]:
# Sample reasoning text
print("Sample Reasoning Texts:")
print("\n" + "="*80 + "\n")

# Show examples with different confidence levels
high_conf = df[df['confidence_level'] >= 0.8].sample(min(3, len(df[df['confidence_level'] >= 0.8])))
med_conf = df[(df['confidence_level'] >= 0.4) & (df['confidence_level'] < 0.6)].sample(min(3, len(df[(df['confidence_level'] >= 0.4) & (df['confidence_level'] < 0.6)])))
low_conf = df[df['confidence_level'] <= 0.2].sample(min(3, len(df[df['confidence_level'] <= 0.2])))

print("HIGH CONFIDENCE (≥0.8) Examples:\n")
for idx, row in high_conf.iterrows():
	print(f"User: {row['channel_title']}")
	print(f"Age: {row['inferred_age_range']}, Gender: {row['inferred_gender']}, Race: {row['inferred_race_ethnicity']}")
	print(f"Confidence: {row['confidence_level']:.2f}")
	print(f"Reasoning: {row['reasoning']}")
	print("\n" + "-"*80 + "\n")

print("MEDIUM CONFIDENCE (0.4-0.6) Examples:\n")
for idx, row in med_conf.iterrows():
	print(f"User: {row['channel_title']}")
	print(f"Age: {row['inferred_age_range']}, Gender: {row['inferred_gender']}, Race: {row['inferred_race_ethnicity']}")
	print(f"Confidence: {row['confidence_level']:.2f}")
	print(f"Reasoning: {row['reasoning']}")
	print("\n" + "-"*80 + "\n")

print("LOW CONFIDENCE (≤0.2) Examples:\n")
for idx, row in low_conf.iterrows():
	print(f"User: {row['channel_title']}")
	print(f"Age: {row['inferred_age_range']}, Gender: {row['inferred_gender']}, Race: {row['inferred_race_ethnicity']}")
	print(f"Confidence: {row['confidence_level']:.2f}")
	print(f"Reasoning: {row['reasoning']}")
	print("\n" + "-"*80 + "\n")

Sample Reasoning Texts:


HIGH CONFIDENCE (≥0.8) Examples:

User: Rachel Z
Age: 45-54, Gender: female, Race: unclear
Confidence: 0.90
Reasoning: The username 'Rachel Z' and profile image show a woman who appears to be in the age range of 45-54. The description indicates a long, accomplished career in music spanning decades, further suggesting this age range. Her gender is clearly female based on the name and presentation. Race/ethnicity is unclear from the available data.

--------------------------------------------------------------------------------

User: Neal “Alexander” Williams
Age: 35-44, Gender: male, Race: black_african_american
Confidence: 0.90
Reasoning: The profile image shows a man appearing to be in his 30s to 40s with features commonly associated with Black/African American ethnicity. The username 'Neal “Alexander” Williams' is typically male. High confidence due to clear visual cues.

--------------------------------------------------------------------------------

Use

In [23]:
# Common words/phrases in reasoning
from collections import Counter
import re

# Extract common phrases
all_reasoning = ' '.join(df['reasoning'].dropna().astype(str))
words = re.findall(r'\b\w+\b', all_reasoning.lower())

# Filter out common stop words
stop_words = {'the', 'a', 'an', 'is', 'are', 'to', 'of', 'in', 'for', 'and', 'or', 'not', 'with', 'as', 'at', 'by', 'from', 'on', 'this', 'that', 'be', 'has', 'have', 'had', 'no'}
filtered_words = [w for w in words if w not in stop_words and len(w) > 3]

word_counts = Counter(filtered_words).most_common(30)
print("\nTop 30 Most Common Words in Reasoning:")
for word, count in word_counts:
	print(f"{word}: {count}")


Top 30 Most Common Words in Reasoning:
demographic: 111
image: 109
username: 101
profile: 100
information: 96
description: 84
country: 77
gender: 77
ethnicity: 66
male: 58
channel: 57
does: 54
human: 50
visual: 49
confidence: 49
data: 46
provide: 45
inference: 45
available: 44
there: 42
race: 42
cues: 40
indicators: 35
name: 35
face: 34
additional: 34
clear: 31
letter: 30
unclear: 30
suggests: 28


## Summary Statistics

In [24]:
print("="*80)
print("DEMOGRAPHIC INFERENCE SUMMARY")
print("="*80)
print(f"\nTotal users analyzed: {len(df)}")
print(f"\nMean confidence: {df['confidence_level'].mean():.3f}")
print(f"Median confidence: {df['confidence_level'].median():.3f}")

print(f"\n--- Age ---")
print(f"Most common: {df['inferred_age_range'].mode()[0]} ({df['inferred_age_range'].value_counts().iloc[0]} users)")
print(f"Unclear rate: {(df['inferred_age_range'] == 'unclear').sum() / len(df) * 100:.2f}%")

print(f"\n--- Gender ---")
print(f"Most common: {df['inferred_gender'].mode()[0]} ({df['inferred_gender'].value_counts().iloc[0]} users)")
print(f"Unclear rate: {(df['inferred_gender'] == 'unclear').sum() / len(df) * 100:.2f}%")

print(f"\n--- Race/Ethnicity ---")
print(f"Most common: {df['inferred_race_ethnicity'].mode()[0]} ({df['inferred_race_ethnicity'].value_counts().iloc[0]} users)")
print(f"Unclear rate: {(df['inferred_race_ethnicity'] == 'unclear').sum() / len(df) * 100:.2f}%")

print(f"\n--- Overall Clarity ---")
print(f"Profiles with all demographics clear: {((df['inferred_age_range'] != 'unclear') & (df['inferred_gender'] != 'unclear') & (df['inferred_race_ethnicity'] != 'unclear')).sum()} ({((df['inferred_age_range'] != 'unclear') & (df['inferred_gender'] != 'unclear') & (df['inferred_race_ethnicity'] != 'unclear')).sum() / len(df) * 100:.2f}%)")
print(f"Profiles with at least one clear inference: {at_least_one_clear} ({at_least_one_clear / len(df) * 100:.2f}%)")
print(f"Profiles with all unclear: {all_unclear} ({all_unclear / len(df) * 100:.2f}%)")
print("\n" + "="*80)

DEMOGRAPHIC INFERENCE SUMMARY

Total users analyzed: 99

Mean confidence: 0.327
Median confidence: 0.200

--- Age ---
Most common: unclear (80 users)
Unclear rate: 80.81%

--- Gender ---
Most common: unclear (60 users)
Unclear rate: 60.61%

--- Race/Ethnicity ---
Most common: unclear (89 users)
Unclear rate: 89.90%

--- Overall Clarity ---
Profiles with all demographics clear: 4 (4.04%)
Profiles with at least one clear inference: 43 (43.43%)
Profiles with all unclear: 56 (56.57%)

